In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance


from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label

print(np.max(label_visiprog))

202.0


In [25]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

[[  0.84626614  -0.0601734   -2.48037165 ...,  11.81214878   4.07557203
   16.28311992]
 [  1.61187253  -0.13726814  -2.84377391 ...,  12.32477588   2.42529768
   16.07555976]
 [  1.73860902  -0.15054939  -2.83881149 ...,  11.88581893   2.37714822
   15.87037336]
 ..., 
 [  1.29760807   0.64739122  -1.62276646 ...,  13.72418317   3.14230751
   16.1829167 ]
 [  1.49290026   0.67626045  -1.12723529 ...,  12.12259606   2.67567087
   16.51988945]
 [  1.67951279   0.64138849  -1.75318638 ...,  12.05057305   3.36389679
   15.61697625]]


In [29]:
accuracy_mat = []
accuracy_vsp = []
accuracy_STSIMM = []

N_material = 61
N_material_train = 10

for fold in range(10):
    try:
        print("Trial {}".format(fold))
        
        train_select = np.zeros_like(label_material).astype(bool)

        idx_train = np.array([])
        for i in range(N_material_train):
            train_class = np.random.randint(0, N_material)
            train_select |= (label_material == train_class)
        
        
        idx_train = np.where(train_select == True)[0]
        idx_test = np.where(train_select == False)[0]
                
        idx_train = idx_train.astype(int)
        idx_test = idx_test.astype(int)
        
        print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

        X_train = raw_feature[idx_train,:]
        X_STSIMM_train = X_STSIMM[idx_train,:]
        Y_VSP_train = label_visiprog[idx_train]
        Y_mat_train = label_material[idx_train]

        X_test = raw_feature[idx_test,:]
        X_STSIMM_test = X_STSIMM[idx_test,:]
        Y_VSP_test = label_visiprog[idx_test]
        Y_mat_test = label_material[idx_test]

        res = leave_one_sample_out(X_STSIMM_test, Y_mat_test)
        accuracy_STSIMM.append(res['accuracy'])
#         print(res['prediction'])
#         print(Y_mat_test)
        
        model = LFDA()
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
        
        model_vsp = LFDA_VISIPROG()
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])

    except Exception as e:
        print(e)

Trial 0
Training shape 694 and test shape 4551
Trial 1
Training shape 791 and test shape 4454
Trial 2
Training shape 640 and test shape 4605
Trial 3
Training shape 916 and test shape 4329
Trial 4
Training shape 828 and test shape 4417
Trial 5
Training shape 824 and test shape 4421
Trial 6
Training shape 640 and test shape 4605
Trial 7
Training shape 736 and test shape 4509
Trial 8
Training shape 736 and test shape 4509
Trial 9
Training shape 883 and test shape 4362


In [30]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSTIMM':accuracy_STSIMM})
display(df)
display(df.mean())

,STSTIMM,material,visiprog
0,0.884201,0.969677,0.954735
1,0.859901,0.962955,0.929951
2,0.881650,0.952443,0.934853
3,0.893971,0.976669,0.965581
4,0.887027,0.977360,0.971021
5,0.886225,0.978059,0.971500
6,0.869055,0.973290,0.963735
7,0.876469,0.963850,0.973608
8,0.880018,0.977157,0.970725
9,0.872306,0.949564,0.961256


STSTIMM     0.879082
material    0.968102
visiprog    0.959697
dtype: float64

In [ ]:
print(model.transformer())